In [ ]:
import csv
from tqdm import tqdm
import networkx as nx
import pandas as pd

In [ ]:
df1 = pd.read_csv('/content/directors_movies.csv',header=0)
df2 = pd.read_csv('/content/final_credits.csv', header=0)

In [ ]:
df1=df1.rename(columns={"title_ids": "title_id"})

In [ ]:
merged_df = pd.merge(df2, df1, on='title_id',how='left')

In [ ]:
merged_df=merged_df.drop(columns=["title"])

In [ ]:
merged_df= merged_df.drop(merged_df[merged_df['Role']=="Directed by"].index)

In [ ]:
merged_df

,Title,Title_URI,Role,Name,Link,Credit,title_id,uri,year,note,director_name,director_id
0,"10,000 BC (2008)",https://www.imdb.com/title/tt0443649/fullcredits/,Visual Effects by,Jim Mitchell,https://www.imdb.com/name/nm0593437/?ref_=ttfc...,visual effects supervisor,tt0443649,https://www.imdb.com/title/tt0443649/?ref_=nm_...,2008,NaN,Roland Emmerich,nm0000386
1,"10,000 BC (2008)",https://www.imdb.com/title/tt0443649/fullcredits/,Sound Department,Jeremy Price,https://www.imdb.com/name/nm1038690/?ref_=ttfc...,supervising sound editor,tt0443649,https://www.imdb.com/title/tt0443649/?ref_=nm_...,2008,NaN,Roland Emmerich,nm0000386
2,"10,000 BC (2008)",https://www.imdb.com/title/tt0443649/fullcredits/,Sound Department,Simon Gershon,https://www.imdb.com/name/nm0314852/?ref_=ttfc...,supervising sound editor,tt0443649,https://www.imdb.com/title/tt0443649/?ref_=nm_...,2008,NaN,Roland Emmerich,nm0000386
3,"10,000 BC (2008)",https://www.imdb.com/title/tt0443649/fullcredits/,Sound Department,Doug Cooper,https://www.imdb.com/name/nm1537681/?ref_=ttfc...,re-recording mixer,tt0443649,https://www.imdb.com/title/tt0443649/?ref_=nm_...,2008,NaN,Roland Emmerich,nm0000386
4,"10,000 BC (2008)",https://www.imdb.com/title/tt0443649/fullcredits/,Sound Department,Chris Burdon,https://www.imdb.com/name/nm0121006/?ref_=ttfc...,re-recording mixer,tt0443649,https://www.imdb.com/title/tt0443649/?ref_=nm_...,2008,NaN,Roland Emmerich,nm0000386
...,...,...,...,...,...,...,...,...,...,...,...,...
29044,Æon Flux (2005),https://www.imdb.com/title/tt0402022/fullcredits/,Writing Credits,Matt Manfredi,https://www.imdb.com/name/nm0542062/?ref_=ttfc...,written by,tt0402022,https://www.imdb.com/title/tt0402022/?ref_=nm_...,2005,NaN,Karyn Kusama,nm0476201
29045,Æon Flux (2005),https://www.imdb.com/title/tt0402022/fullcredits/,Visual Effects by,Syd Dutton,https://www.imdb.com/name/nm0244956/?ref_=ttfc...,visual effects supervisor,tt0402022,https://www.imdb.com/title/tt0402022/?ref_=nm_...,2005,NaN,Karyn Kusama,nm0476201
29046,Æon Flux (2005),https://www.imdb.com/title/tt0402022/fullcredits/,Visual Effects by,Bryan Hirota,https://www.imdb.com/name/nm0386415/?ref_=ttfc...,visual effects supervisor,tt0402022,https://www.imdb.com/title/tt0402022/?ref_=nm_...,2005,NaN,Karyn Kusama,nm0476201
29047,Æon Flux (2005),https://www.imdb.com/title/tt0402022/fullcredits/,Sound Department,Jon Taylor,https://www.imdb.com/name/nm0852656/?ref_=ttfc...,re-recording mixer,tt0402022,https://www.imdb.com/title/tt0402022/?ref_=nm_...,2005,NaN,Karyn Kusama,nm0476201


In [ ]:
G = nx.DiGraph()

In [ ]:
roles = merged_df["Role"].tolist()

In [ ]:
G.add_nodes_from(merged_df['Name'])
G.add_nodes_from(merged_df['director_name'])

for index, row in merged_df.iterrows():
  if G.has_edge(row[3], row[10]):
      # edge already exists, increase weight by one
      G[row[3]][row[10]]['weight'] += 1
  else:
      # add new edge with weight 1
      G.add_edge(row[3], row[10], weight = 1)

In [ ]:
nx.write_gexf(G, "/content/director_crew_weighted.gexf")

In [ ]:
G2 = nx.MultiDiGraph()
G2.add_nodes_from(merged_df['Name'])
G2.add_nodes_from(merged_df['director_name'])
edge_number=0
for index, row in merged_df.iterrows():
  G2.add_edge(row[3], row[10], key=edge_number)
  attrs = {(row[3], row[10], edge_number): {"Year": row[8], "Role": row[2]}}
  nx.set_edge_attributes(G2, attrs)
  edge_number += 1


In [ ]:
nx.write_gexf(G2, "/content/attributes.gexf")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files
files.download('attributes.gexf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>